<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:51952 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 16.96 GB


## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [4]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [5]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Charlie", "transactions": [{"transaction-id": 885, "amount": 673}, {"transaction-id": 1169, "amount": 666}, {"transaction-id": 1901, "amount": 614}, {"transaction-id": 2315, "amount": 608}, {"transaction-id": 3208, "amount": 725}, {"transaction-id": 4198, "amount": 576}, {"transaction-id": 6278, "amount": 637}, {"transaction-id": 6392, "amount": 786}, {"transaction-id": 6693, "amount": 640}, {"transaction-id": 7074, "amount": 593}, {"transaction-id": 7536, "amount": 655}, {"transaction-id": 10001, "amount": 640}, {"transaction-id": 11630, "amount": 679}, {"transaction-id": 11990, "amount": 514}, {"transaction-id": 12694, "amount": 645}, {"transaction-id": 12816, "amount": 612}, {"transaction-id": 13047, "amount": 711}, {"transaction-id": 13063, "amount": 588}, {"transaction-id": 13070, "amount": 587}, {"transaction-id": 14133, "amount": 680}, {"transaction-id": 14147, "amount": 575}, {"transaction-id": 15875, "amount": 687}, {"transaction-id": 16141, "amount": 693}

In [8]:
# Requires `s3fs` library
# each partition is a remote CSV text file
import s3fs
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [10]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [11]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [12]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Charlie", "transactions": [{"transaction-id": 885, "amount": 673}, {"transaction-id": 1169, "amount": 666}, {"transaction-id": 1901, "amount": 614}, {"transaction-id": 2315, "amount": 608}, {"transaction-id": 3208, "amount": 725}, {"transaction-id": 4198, "amount": 576}, {"transaction-id": 6278, "amount": 637}, {"transaction-id": 6392, "amount": 786}, {"transaction-id": 6693, "amount": 640}, {"transaction-id": 7074, "amount": 593}, {"transaction-id": 7536, "amount": 655}, {"transaction-id": 10001, "amount": 640}, {"transaction-id": 11630, "amount": 679}, {"transaction-id": 11990, "amount": 514}, {"transaction-id": 12694, "amount": 645}, {"transaction-id": 12816, "amount": 612}, {"transaction-id": 13047, "amount": 711}, {"transaction-id": 13063, "amount": 588}, {"transaction-id": 13070, "amount": 587}, {"transaction-id": 14133, "amount": 680}, {"transaction-id": 14147, "amount": 575}, {"transaction-id": 15875, "amount": 687}, {"transaction-id": 16141, "amount": 693}

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [13]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Charlie',
  'transactions': [{'transaction-id': 885, 'amount': 673},
   {'transaction-id': 1169, 'amount': 666},
   {'transaction-id': 1901, 'amount': 614},
   {'transaction-id': 2315, 'amount': 608},
   {'transaction-id': 3208, 'amount': 725},
   {'transaction-id': 4198, 'amount': 576},
   {'transaction-id': 6278, 'amount': 637},
   {'transaction-id': 6392, 'amount': 786},
   {'transaction-id': 6693, 'amount': 640},
   {'transaction-id': 7074, 'amount': 593},
   {'transaction-id': 7536, 'amount': 655},
   {'transaction-id': 10001, 'amount': 640},
   {'transaction-id': 11630, 'amount': 679},
   {'transaction-id': 11990, 'amount': 514},
   {'transaction-id': 12694, 'amount': 645},
   {'transaction-id': 12816, 'amount': 612},
   {'transaction-id': 13047, 'amount': 711},
   {'transaction-id': 13063, 'amount': 588},
   {'transaction-id': 13070, 'amount': 587},
   {'transaction-id': 14133, 'amount': 680},
   {'transaction-id': 14147, 'amount': 575},
   {'transaction-id

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [15]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 10,
  'name': 'Alice',
  'transactions': [{'transaction-id': 131, 'amount': 84},
   {'transaction-id': 211, 'amount': 87},
   {'transaction-id': 442, 'amount': 43},
   {'transaction-id': 864, 'amount': 86},
   {'transaction-id': 1522, 'amount': 103},
   {'transaction-id': 1894, 'amount': -25},
   {'transaction-id': 2044, 'amount': 0},
   {'transaction-id': 2062, 'amount': 119},
   {'transaction-id': 3539, 'amount': 116},
   {'transaction-id': 3856, 'amount': 227},
   {'transaction-id': 4028, 'amount': 235},
   {'transaction-id': 4489, 'amount': 20},
   {'transaction-id': 4548, 'amount': 251},
   {'transaction-id': 4819, 'amount': 162},
   {'transaction-id': 6108, 'amount': 269},
   {'transaction-id': 6322, 'amount': 213},
   {'transaction-id': 6558, 'amount': 48},
   {'transaction-id': 7341, 'amount': 87},
   {'transaction-id': 7568, 'amount': 341},
   {'transaction-id': 8193, 'amount': 95},
   {'transaction-id': 8208, 'amount': 55},
   {'transaction-id': 9007, 'amount': 168},


In [16]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 170},
 {'name': 'Alice', 'count': 552},
 {'name': 'Alice', 'count': 491},
 {'name': 'Alice', 'count': 63},
 {'name': 'Alice', 'count': 182})

In [18]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(170, 552, 491, 63, 182)

In [15]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

265.7616

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [16]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 131, 'amount': 84},
  {'transaction-id': 211, 'amount': 87},
  {'transaction-id': 442, 'amount': 43},
  {'transaction-id': 864, 'amount': 86},
  {'transaction-id': 1522, 'amount': 103},
  {'transaction-id': 1894, 'amount': -25},
  {'transaction-id': 2044, 'amount': 0},
  {'transaction-id': 2062, 'amount': 119},
  {'transaction-id': 3539, 'amount': 116},
  {'transaction-id': 3856, 'amount': 227},
  {'transaction-id': 4028, 'amount': 235},
  {'transaction-id': 4489, 'amount': 20},
  {'transaction-id': 4548, 'amount': 251},
  {'transaction-id': 4819, 'amount': 162},
  {'transaction-id': 6108, 'amount': 269},
  {'transaction-id': 6322, 'amount': 213},
  {'transaction-id': 6558, 'amount': 48},
  {'transaction-id': 7341, 'amount': 87},
  {'transaction-id': 7568, 'amount': 341},
  {'transaction-id': 8193, 'amount': 95},
  {'transaction-id': 8208, 'amount': 55},
  {'transaction-id': 9007, 'amount': 168},
  {'transaction-id': 9673, 'amount': 209},
  {'transaction-id': 10526

In [21]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 131, 'amount': 84},
 {'transaction-id': 211, 'amount': 87},
 {'transaction-id': 442, 'amount': 43})

In [22]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(84, 87, 43)

In [19]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

700.9501779038055

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [31]:
b.compute()

['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank']

In [29]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [32]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [33]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [34]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [35]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

KeyboardInterrupt: 

In [38]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 1250), ('Bob', 900), ('Charlie', 700), ('Dan', 1100), ('Edith', 1100), ('Frank', 800), ('George', 1049), ('Hannah', 932), ('Ingrid', 1149), ('Jerry', 800), ('Kevin', 800), ('Laura', 800), ('Michael', 1050), ('Norbert', 834), ('Oliver', 950), ('Patricia', 750), ('Quinn', 750), ('Ray', 1050), ('Sarah', 1000), ('Tim', 1000), ('Ursula', 900), ('Victor', 850), ('Wendy', 1145), ('Xavier', 1747), ('Yvonne', 949), ('Zelda', 600)]
Wall time: 1.72 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [39]:
# Your code here...
def amount()

result = js.foldby(key='name',
                   binop=)

TypeError: foldby() missing 1 required positional argument: 'binop'

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [ ]:
df1 = js.to_dataframe()
df1.head()

This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [ ]:
%time df1.groupby('name').id.count().compute().head()

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [ ]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

In [ ]:
df = transactions.to_dataframe()
df.head()

In [ ]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [ ]:
client.shutdown()